# Cuaderno de cálculos y gráficas

In [1]:
# Import modules
import ipywidgets
import numpy as np
import pandas as pd
import importlib
import logging
import glob
import os

# All the stuffs for the processing of MI
import processing as pr

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# Signal processing
import scipy.signal as snl
from scipy.io import wavfile
import scipy.stats as stats

# Logging
import logging
logging.disable(logging.CRITICAL)

 2020-06-10 08:31:51,591 - DEBUG - Define classes and functions.
 2020-06-10 08:31:51,594 - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


### Load MI_diff.dat

In [5]:
midiff = open("MI_diff.dat", "r")
midiff = eval(midiff.readline())
mirel = open("MI_rel.dat", "r")
mirel = eval(mirel.readline())

In [11]:
df = pd.DataFrame(mirel)

In [12]:
df.to_excel("mirel.xlsx")

### File paths

In [2]:
mi_folder = "../Data/mutualinfo/"
mi_files = glob.glob(mi_folder + "ds1/*.mi")
mi_files.sort()
ds8_files = glob.glob(mi_folder + "ds8/*.mi")
ds8_files.sort()

To open the files with the MI data already averaged
```
with open("MI_rel.dat", "r") as f:
    line = f.readline()
    MI_rel = eval(line)
with open("MI_rel_ds8.dat", "r") as f:
    line = f.readline()
    MI_rel_ds8 = eval(line)
    
bands = list(MI_rel.keys())
channels = list(MI_rel["delta"].keys())
```

### Get MI info from files
We've already calculated all the MI values. Now we will take those values and do some statistical calculations on them.

In [4]:
importlib.reload(pr)
cumdata = {}
total = len(mi_files)
current = 0
for file in mi_files:
    data = pr.MutualInformation()
    data.load(file_path = file)
    # name = os.path.split(file)[1].split(".")[0] + "_ds8.mi
    # folder = os.path.split(os.path.split(file)[0])[0] + "/ds8/"
    # path = folder + name
    # datads8 = pr.MutualInformation(1,2,3,4,5,True,path)
    
    try:
        bands
    except NameError:
        bands = list(data.mi.keys())
        channels = list(data.mi[bands[0]].keys()) 
        
    if cumdata == {}:
        for band in bands:
            cumdata[band] = {}
            for channel in channels:
                cumdata[band][channel] = {"face": [], "noface": []}

    for band in bands:
        for channel in channels:
            if data.face_nonscrambled:
                cumdata[band][channel]["face"].append(data.mi[band][channel])
            else:
                cumdata[band][channel]["noface"].append(data.mi[band][channel])
    current += 1
    if current%1000 == 0 or current == total:
        print("Gone through " + str(current/total *100))

Gone through 18.49796522382538
Gone through 36.99593044765076
Gone through 55.49389567147613
Gone through 73.99186089530151
Gone through 92.48982611912689
Gone through 100.0


### Prepare parametric and non-parametric tests (one way ANOVA and Kruskal-Wallis ANOVA)

In [8]:
fandp_anova = {}
for band in bands:
    fandp_anova[band] = {}
    for channel in channels:
        fandp_anova[band][channel] = stats.f_oneway(cumdata[band][channel]["face"], cumdata[band][channel]["noface"])
fandp_kruskal = {}
for band in bands:
    fandp_kruskal[band] = {}
    for channel in channels:
        fandp_kruskal[band][channel] = stats.kruskal(cumdata[band][channel]["face"], cumdata[band][channel]["noface"])

savefile = open("data.csv", "w")
savefile.write("band, channel, owanova_F, owanova_p, kruskal_F, kruskal_p\n")

for band in bands:
    for channel in channels:
        str0 = str(band) + ", " + str(channel) + ", "
        list1 = list(fandp_anova[band][channel])
        str1 = str(list1[0]) + ", " + str(list1[1]) + ", "
        list2 = list(fandp_kruskal[band][channel])
        str2 = str(list2[0]) + ", " + str(list2[1]) + "\n"
        strend = str0 + str1 + str2
        savefile.write(strend)
        
        #print("--- BAND " + band + " CHANNEL " + channel + " ---")
        #print("OWANOVA: " + str(fandp_anova[band][channel]))
        #print("KRUSKAL: " + str(fandp_kruskal[band][channel]))

In [37]:
vals = [float(value) for value in cumdata["gamma"]["Fpz"]["noface"]]

In [39]:
vals = np.array(vals)
vals = (vals - vals.min())/vals.max()

In [44]:
hist = np.histogram(vals, 4)

In [45]:
hist

(array([  16,   73,  334, 2279]),
 array([0.        , 0.00498854, 0.00997709, 0.01496563, 0.01995417]))

In [ ]:
plt.plot()

## Plotting part

In [ ]:
fig = plt.figure()

bands = list(MI_rel.keys())
k = bands[0]
labels = list(MI_rel[k].keys())
mi_rel_vals = list(MI_rel[k].values())
ds8_rel_vals = list(MI_rel_ds8[k].values())

In [ ]:
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, mi_rel_vals, width, label='No downsampling')
rects2 = ax.bar(x + width/2, ds8_rel_vals, width, label='Downsampling')

ax.set_ylabel('Relative Mutual Information')
ax.set_title('Comparison between downsampled and non-downsampled MI values')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


fig.tight_layout()

plt.show()